In [4]:
import tensorflow as tf
import numpy as np
import os
import cv2
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import data_prj as dp

batch_size = 64
epochs = 7

learning_rate = .001

panneaux_id, panneaux_image = dp.get_panneaux(dp.rep_panneaux, dp.panneau_shape)

model = dp.create_model1(dp.panneau_shape)

"""
model = tf.keras.Sequential()

model.add(layers.Input(shape = (panneau_shape[0], panneau_shape[1], 3), dtype='float32'))

model.add(layers.Conv2D(64, 3, strides=1))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPool2D(pool_size=2, strides=2))

model.add(layers.Conv2D(128, 3, strides=1))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPool2D(pool_size=2, strides=2))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(1, activation='sigmoid'))
"""


tab_images = np.array([]).reshape(0, dp.panneau_shape[0], dp.panneau_shape[1], 3)

nbre_images = 0
for image in panneaux_image :
    gen_images = dp.generate_images(image, 12500)
    tab_images = np.concatenate([tab_images, gen_images])
    nbre_images += len(gen_images)
tab_labels = np.full(nbre_images, 1)
print("Nombre de paneaux : ", nbre_images)


files = os.listdir(dp.rep_panneaux_diff)
nbre_images = 0
for file in files:
    path = os.path.join(dp.rep_panneaux_diff, file)
    image = cv2.resize(cv2.imread(path), dp.panneau_shape)
    gen_images = dp.generate_images(image, 1250)
    tab_images = np.concatenate([tab_images, gen_images])
    nbre_images += len(gen_images)
tab_labels = np.concatenate([tab_labels, np.full(nbre_images, 0)])
print("Nombre de paneaux differents : ", nbre_images)

nbre_non_panneau = int(len(tab_images)/2)
nbre_images = 0
gen_images = list()
for no_image in random.sample(range(0, 100000), nbre_non_panneau):
    file = dp.rep_non_panneaux + "/{:d}.png".format(no_image)
    image = cv2.resize(cv2.imread(file), dp.panneau_shape)
    gen_images.append(image)
    nbre_images += 1
tab_images = np.concatenate([tab_images, gen_images])
tab_labels = np.concatenate([tab_labels, np.full(nbre_images, 0)])
print("Nombre de non paneaux : ", nbre_images)


tab_images = np.array(tab_images, dtype=np.float32) / 255
tab_labels = np.array(tab_labels).reshape([-1, 1])

tab_images, tab_labels = shuffle(tab_images, tab_labels)
train_images, test_images, train_labels, test_labels = train_test_split(tab_images, tab_labels, test_size=0.15)

train_data = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_data = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

train_batch = train_data.shuffle(len(train_data)).batch(batch_size)
test_batch = test_data.shuffle(len(test_data)).batch(batch_size)

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

history = model.fit(train_batch, epochs = epochs, validation_data = test_batch)

model.save("model1.h5")

Nombre de paneaux :  75000
Nombre de paneaux differents :  25000
Nombre de non paneaux :  50000
Epoch 1/7
1993/1993 [==============================] - 40s 17ms/step - loss: 0.0937 - binary_accuracy: 0.9643 - val_loss: 0.1387 - val_binary_accuracy: 0.9528
Epoch 2/7
1993/1993 [==============================] - 35s 17ms/step - loss: 0.0419 - binary_accuracy: 0.9846 - val_loss: 0.1013 - val_binary_accuracy: 0.9615
Epoch 3/7
1993/1993 [==============================] - 35s 17ms/step - loss: 0.0332 - binary_accuracy: 0.9879 - val_loss: 0.0401 - val_binary_accuracy: 0.9850
Epoch 4/7
1993/1993 [==============================] - 34s 17ms/step - loss: 0.0285 - binary_accuracy: 0.9897 - val_loss: 0.3008 - val_binary_accuracy: 0.9367
Epoch 5/7
1993/1993 [==============================] - 35s 17ms/step - loss: 0.0188 - binary_accuracy: 0.9936 - val_loss: 0.0342 - val_binary_accuracy: 0.9860
Epoch 6/7
1993/1993 [==============================] - 35s 17ms/step - loss: 0.0174 - binary_accuracy: 0.9934